#서울시 전체는 제낀다(나중에 sum함수를 통해 계산값으로 집어넣기)
#함수 2개 사용
1) selenium을 사용, 기준분기/생활밀접업종/점포수를 세팅하고 검색 버튼을 눌러주는 함수
for문 한 개 사용
생활밀접업종(L)은 외식업으로 고정
정보분류는 '점포수'로 고정
변수 1 : 생활밀접업종(M)
커피-음료 CS100010
호프-간이주점 CS100009
분식전문점 CS100008
치킨전문점 CS100007
패스트푸드점 CS100006
제과점 CS100005
양식음식점 CS100004
일식음식점 CS100003
중식음식점 CS100002
한식음식점 CS100001
이렇게 돌림
변수 2 : 연도
변수 3 : 분기
2017 1 2 3 4
2018 1 2 3 4
2019 1 2 3 4
2020 1 2 3 4
for문 세개 필요????? 에반데

2) driver.page_source에서 연도를 넘기고 행을 내리는 함수(for문 두 개 사용)
1. 연도를 넘기는 for문
전체 점포수~일반 점포수 넣는 부분에서 인덱스값이
234 > 567 > 8910
으로 바뀐다. (한 번 돌아갈 때마다 기존 인덱스값+3이 된다.)

2. 행을 내리는 for문
15부터 464까지 돌린다 (맨 마지막 행인 둔촌2동 = 463번)
= 행정구역(tr[12].find_all('th')[2].text)
행정구역[-1]이 '구'라면 '구' column에 넣고 '동'에는 '전체'라고 기입
'구' 정보를 어디에 저장을 해두기
행정구역[-1]이 '동'이라면 저장해온 구 정보를 불러와서 '구' column에 넣고 행정구역[-1]을 '동' column에 넣기

= 생활밀접업종(tr[12].find_all('th')[3].text)
생활밀접업종에 넣기
= 전체 점포수 ~ 일반 점포수(tr[14].find_all('td')[2~4, 5~7, 8~10])
각각의 리스트에 넣기

#각각의 column을 빈 리스트 형태로 만들어 놓고 차례대로 추가한다.
#나중에 크롤링이 끝나면 리스트를 종합해서 데이터 프레임 형식으로 만든다.
#데이터프레임을 csv파일로 만들어서 저장한다.

##작업 순서 2 -> 1

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json
import requests
import re
from tqdm.notebook import tqdm
import pandas

In [2]:
# url 열기
driver = webdriver.Chrome('../data/chromedriver')
url = "https://golmok.seoul.go.kr/regionAreaAnalysis.do"
driver.get(url)

# 비회원 / 회원 중 비회원 선택하기
guest = driver.find_element_by_css_selector('button.btn.btn-guest').click()

# 실제 작업공간
gu = []
dong = []
indu = []
year = []
qu = []
fran_cnt = []
normal_cnt = []

def crawl_every():
    # selectbox 선택
    # 무조건 고정시키는 것
    for i in tqdm(['2017','2018','2019','2020']):
        selectInduL = driver.find_element_by_xpath('//*[@id="induL"]/option[2]').click()
        time.sleep(1)
        selectInfo = driver.find_element_by_xpath('//*[@id="infoCategory"]/option[2]').click()
        time.sleep(1)
        for j in ['1','2','3','4']:
            if (i == '2020' and j == '2'):
                break
            else:    
                selectYear = driver.find_element_by_css_selector('#selectYear').send_keys(i)
                selectQu = driver.find_element_by_css_selector('#selectQu').send_keys(j)
                time.sleep(1)            
                for m in ['2','3','4','5','6','7','8','9','10','11']:
                    selectInduM = driver.find_element_by_xpath('//*[@id="induM"]/option['+ m +']').click()
                    time.sleep(1)
                    selectClear = driver.find_element_by_css_selector('#presentSearch').click()
                    time.sleep(1)
                    crawl_one()


def crawl_one():
    soup = BeautifulSoup(driver.page_source, 'lxml')
    tr = soup.find_all('tr')

    # '2015년 1분기'를 ['2015', '1']로 바꿔주는 함수
    def toNum(year):
        pattern = re.compile('[0-9]+')
        a = pattern.findall(year)
        return a

    # 2015 - 2016 - 2017로 전환
    def YandQ(j):
        if j==3:
            return toNum(tr[12].find_all('th')[2].text)
        elif j==6:
            return toNum(tr[12].find_all('th')[3].text)
        elif j==9:
            return toNum(tr[12].find_all('th')[4].text)

    gu_temp = ""

    for j in [3,6,9]:
        for i in range(15,464):
            td = tr[i].find_all('td')
            district = td[0].text
            year.append(YandQ(j)[0])
            qu.append(YandQ(j)[1])
            if district[-1] == '구':
                gu_temp = district.replace('\xa0',"")
                gu.append(gu_temp)
                dong.append('전체')
                indu.append(td[1].text)
                fran_cnt.append(td[j].text)
                normal_cnt.append(td[j+1].text)
            else:
                gu.append(gu_temp)
                dong.append(district)
                indu.append(td[1].text)
                fran_cnt.append(td[j].text)
                normal_cnt.append(td[j+1].text)

crawl_every()

print(len(gu))
print(len(dong))
print(len(indu))
print(len(fran_cnt))
print(len(normal_cnt))
print(len(year))
print(len(qu))


175110
175110
175110
175110
175110
175110
175110


In [3]:
import pandas as pd
df = pd.DataFrame({'year':year, 'quarter':qu, 'gu':gu, 'dong':dong, 'industry':indu, 'franchise':fran_cnt, 'normal_store':normal_cnt})

In [4]:
df

,year,quarter,gu,dong,industry,franchise,normal_store
0,2015,1,종로구,전체,외식업/한식음식점,208,"2,887"
1,2015,1,종로구,청운효자동,외식업/한식음식점,5,76
2,2015,1,종로구,사직동,외식업/한식음식점,33,391
3,2015,1,종로구,삼청동,외식업/한식음식점,2,73
4,2015,1,종로구,부암동,외식업/한식음식점,2,41
...,...,...,...,...,...,...,...
175105,2020,1,강동구,성내2동,외식업/커피-음료,10,46
175106,2020,1,강동구,성내3동,외식업/커피-음료,13,59
175107,2020,1,강동구,길동,외식업/커피-음료,23,55
175108,2020,1,강동구,둔촌1동,외식업/커피-음료,0,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175110 entries, 0 to 175109
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          175110 non-null  object
 1   quarter       175110 non-null  object
 2   gu            175110 non-null  object
 3   dong          175110 non-null  object
 4   industry      175110 non-null  object
 5   franchise     175110 non-null  object
 6   normal_store  175110 non-null  object
dtypes: object(7)
memory usage: 9.4+ MB


In [6]:
df.to_excel('franchise.xlsx')